In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# NiN 中的mlpconv块
def mlpconv(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding), nn.ReLU(),
        # 卷积层后接两层1×1的卷积层，用于添加非线性并帮助特征提取
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),  
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU()
    )

In [3]:
net = nn.Sequential(
    mlpconv(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    mlpconv(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    mlpconv(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),  # 原论文中有提到加了dropout之后准确率会上升。。。
    # 标签类别数是10
    mlpconv(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),  # 全局平均池化直接转换feature map为categories
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten()
)

In [4]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


In [ ]:
# 由于计算量过大，我的CPU瑟瑟发抖，所以就不具体训练了！